In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/la-weights-final/la_final.h5
/kaggle/input/sa-weights-final/sa_final.h5
/kaggle/input/tensorflow2-question-answering/simplified-nq-test.jsonl
/kaggle/input/tensorflow2-question-answering/simplified-nq-train.jsonl
/kaggle/input/tensorflow2-question-answering/sample_submission.csv
/kaggle/input/berthub/saved_model.pb
/kaggle/input/berthub/assets/vocab.txt
/kaggle/input/berthub/variables/variables.index
/kaggle/input/berthub/variables/variables.data-00000-of-00001
/kaggle/input/weights-test/long_answer_final.h5
/kaggle/input/zbertjointbaseline/vocab-nq.txt
/kaggle/input/zbertjointbaseline/model_cpkt-1.data-00000-of-00002
/kaggle/input/zbertjointbaseline/model_cpkt-1.data-00001-of-00002
/kaggle/input/zbertjointbaseline/model_cpkt-1.index
/kaggle/input/zbertjointbaseline/nq-train.tfrecords
/kaggle/input/zbertjointbaseline/bert_utils.py
/kaggle/input/zbertjointbaseline/tokenization.py
/kaggle/input/zbertjointbaseline/nq-test.tfrecords
/kaggle/input/zbertjointbaseline/modeling.p

In [16]:
from tqdm import tqdm
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras.models import Model
from transformers import BertTokenizer
import sys
sys.path.append('/kaggle/input/zbertjointbaseline/')
import modeling 
import time

test_path = "/kaggle/input/tensorflow2-question-answering/simplified-nq-test.jsonl"
sub_path = "/kaggle/input/tensorflow2-question-answering/sample_submission.csv"
#la_weights_path = "/kaggle/input/la-weights-final/la_final.h5"
la_weights_path = "/kaggle/input/weights-test/long_answer_final.h5"
sa_weights_path = "/kaggle/input/sa-weights-final/sa_final.h5"
vocab_path = "/kaggle/input/berthub/assets/vocab.txt"


tokenizer = BertTokenizer(vocab_file=vocab_path)

path = test_path

def get_cosine(qns, ans):
    h = TfidfVectorizer()
    h = h.fit([qns+ans])
    vec1 = h.transform([qns])
    vec2 = h.transform([ans])
    c = cosine_similarity(vec1,vec2)
    return float(c[0])


counter = 0
cols = ['id', 'questions', 'candidate', 'span', 'jaq_sim']

def jac(df):
    df['jaq_sim'] =""
    for i in df.index:
        df.loc[i,'jaq_sim'] = get_jaccard_sim(df.loc[i,'question'],df.loc[i,'candidate'])
    df.sort_values('jaq_sim',ascending=False,inplace=True)
    return df.head(25)

def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))
with open(path, 'rt') as json_file:
    counter += 1
    #ids = []
    #qn =[]
    #ans = []
    #span = []
    #cos_sim = []
    doc_text = []
    qid = []
    df = pd.DataFrame()
    for line in tqdm(json_file):
        counter +=1
        #if counter == 10:
        #    break
        t = json.loads(line)
        text = t['document_text']
        doc_text.append(text)
        idn = t['example_id']
        qid.append(idn)
        q = t['question_text']
        ids = []
        qn =[]
        ans = []
        span = []
        jaq_sim = []
        for p in t['long_answer_candidates']:
            ans1 = p['start_token']
            ans2 = p['end_token']
            sp_doc = t['document_text'].split(" ")
            span1 = str(ans1)+":"+str(ans2)
            answer = " ".join(sp_doc[ans1:ans2])
            jac = get_jaccard_sim(q,answer)
            ids.append(idn)
            qn.append(q)
            ans.append(answer)
            span.append(span1)
            jaq_sim.append(jac)
        temp_df = pd.DataFrame(list(zip(ids,qn,ans,span,jaq_sim)),columns =cols)
        temp_df.sort_values("jaq_sim",ascending=False,inplace=True)
        temp_df = temp_df.head(25)
        #print(temp_df)
        df = pd.concat([df,temp_df])
        
        
    #df = pd.DataFrame()
    #df['id'] = ids
    #df['question'] = qn
    #df['answer'] =ans
    #df['span'] = span
    #df['cos_sim'] = cos_sim
    
df = df.rename({"question":'questions','answer':'candidate'},axis=1)
end = time.time()
#print(end-start)
df.reset_index(inplace=True,drop=True)
doc_text = pd.DataFrame({"id":qid,'text':doc_text})

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def tester(x,df,y=256,p=False,train=True):
    input_ids = []
    input_masks = []
    input_segments = []

    ml = y
    for i in df.head(x).index:
        ans = strip_tags(str(df.loc[i,'candidate']))
        ans = ans.replace(".","[SEP]")
        ans = ans.replace('""', '')
        ans = ans.replace('``','')
        ans = ans.replace("''", "")
        #ans = "[CLS] " + ans
        ans = " ".join(ans.split())

        qn = strip_tags(str(df.loc[i,'questions']))
        qn = qn.replace(".","[SEP]")
        qn = "[CLS] " + qn + "[CLS]"
        fn = qn + " " + ans
        if p:
            print(fn)
        tokens = tokenizer.tokenize(fn)[:ml]
        input_ids.append(get_ids(tokens,tokenizer,ml))
        input_masks.append(get_masks(tokens,ml))
        input_segments.append(get_segments(tokens,ml))

    if train:
        inputs = [tf.convert_to_tensor(input_ids,dtype=float),tf.convert_to_tensor(input_masks,dtype=float),tf.convert_to_tensor(input_segments,dtype=float)
                ]

        #output = tf.convert_to_tensor(df.head(x)['target'],dtype=tf.int32)
        #return inputs,output

        output = tf.one_hot(tf.convert_to_tensor(df.head(x)['target'],dtype=tf.int32),depth=2)
        return inputs,output
    else:
        inputs = [tf.convert_to_tensor(input_ids,dtype=float),tf.convert_to_tensor(input_masks,dtype=float),tf.convert_to_tensor(input_segments,dtype=float)
                ]
        return inputs
    
inp = tester(len(df),df,train=False)


def mk_model(path):
    max_seq_length = 256  # Your choice here.
    input_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                           name="input_ids")
    input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_mask")
    segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                        name="segment_ids")
    bert_layer1 = hub.KerasLayer("/kaggle/input/berthub",trainable=True,name="bert")
    #bert_layer1 = bert_layer
    pooled_output, sequence_output = bert_layer1([input_ids, input_mask, segment_ids])
    x = tf.keras.layers.Dense(128,activation=tf.nn.relu,name='combined_layer_2',kernel_initializer=tf.random_normal_initializer)(pooled_output)
    logits = tf.keras.layers.Dense(2,activation='softmax')(x)
    model = Model(inputs=[input_ids,input_mask,segment_ids], outputs=logits)
    model.load_weights(path)
    return model

model = mk_model(la_weights_path)

def predict(model,df):
    y_preds = model.predict([i for i in inp])
    y_probs = [max(y) for y in y_preds]
    y = [np.argmax(y) for y in y_preds]
    df['predictions'] = y
    df['probs'] = y_probs
    return df

#sa_model = mk_sa_model(sa_weights_path)

7it [00:00, 17.65it/s]


In [3]:
p = predict(model,df)
p = p.sort_values('probs',ascending=False)
p = p.loc[p['predictions']==1]
p= p.drop_duplicates(subset='id',keep='first')

In [4]:
import nltk
cols = ['id',"question",'candidate','long_answer','target']
def created_sa_cands(df,x=False,y=False):
    ndf = pd.DataFrame(columns=cols)
    qids = []
    qs = []
    las =[]
    cands = []

    if x:
        df = df.loc[x:y]
    for i in tqdm(df.index):
        df.loc[i,'candidate'] = strip_tags(str(df.loc[i,'candidate']))

        qid = df.loc[i,'id']
        q = df.loc[i,'questions']
        la = df.loc[i,'candidate']
        
        sents = split_into_sentences(la)
        for sent in sents:
            qids.append(qid)
            qs.append(q)
            las.append(la)
            cands.append(sent)
            
        nes = preprocess(la)
        text = ""
        temp = []
        for ne in nes:
            
            if ne[1] in ["NNP","CD"]:
                text += ne[0]
                text += " "
            elif ne[1] == "CC" and last_e in ["NNP","CD"]:
                text += ne[0]
                text += " "
            else:
                if text and text not in temp:
                    qids.append(qid)
                    qs.append(q)
                    las.append(la)
                    cands.append(text)
                    temp.append(text)
                    
                    text = ""
            last_e = ne[1]
        qids.append(qid)
        qs.append(q)
        las.append(la)
        cands.append("Yes")
       
        qids.append(qid)
        qs.append(q)
        las.append(la)
        cands.append("No")
        
        
    ndf['id'] = qids
    ndf['question'] = qs
    ndf['long_answer'] = las
    ndf['candidate'] = cands
    
    return ndf

import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [5]:
sa = created_sa_cands(p)

100%|██████████| 8/8 [00:00<00:00, 17.67it/s]


In [13]:
def tester(df,y=256):
    #del df
    df.reset_index(drop=True,inplace=True)
    input_ids = []
    input_masks = []
    input_segments = []
    p = list(df.index)
    ml = y
    for i in tqdm(p):
        ans = strip_tags(str(df.loc[i,'long_answer']))
        ans = ans.replace(".","[SEP]")
        ans = ans.replace('""', '')
        ans = ans.replace('``','')
        ans = ans.replace("''", "")
        #ans = "[CLS] " + ans
        ans = " ".join(ans.split())

        qn = strip_tags(str(df.loc[i,'question']))
        qn = qn.replace(".","[SEP]")
        qn = "[CLS] " + qn + "[CLS]"
        cn = str(df.loc[i,'candidate'])
        fn = qn + " " + cn +"[CLS]" + ans
        #print(fn)
        toks = tokenizer.tokenize(fn)
        tokens = toks[:ml]
        input_ids.append(get_ids(tokens,tokenizer,ml))
        input_masks.append(get_masks(tokens,ml))
        input_segments.append(get_segments(tokens,ml))


    inputs = [tf.convert_to_tensor(input_ids,dtype=float),tf.convert_to_tensor(input_masks,dtype=float),tf.convert_to_tensor(input_segments,dtype=float)
            ]

    
    return inputs

sa_inps = tester(df =sa.head())



100%|██████████| 5/5 [00:00<00:00, 314.40it/s]

[CLS] when was i want to sing in opera written[CLS] Wilkie Bard ( born William August Smith ) ( 19 March 1874 -- 5 May 1944 ) was a popular British vaudeville and music hall entertainer and recording artist at the beginning of the 20th century .[CLS]Wilkie Bard ( born William August Smith ) ( 19 March 1874 -- 5 May 1944 ) was a popular British vaudeville and music hall entertainer and recording artist at the beginning of the 20th century [SEP] He is best known for his songs I Want to Sing in Opera and The Night Watchman [SEP]
[CLS] when was i want to sing in opera written[CLS] He is best known for his songs `` I Want to Sing in Opera '' and `` The Night Watchman '' .[CLS]Wilkie Bard ( born William August Smith ) ( 19 March 1874 -- 5 May 1944 ) was a popular British vaudeville and music hall entertainer and recording artist at the beginning of the 20th century [SEP] He is best known for his songs I Want to Sing in Opera and The Night Watchman [SEP]
[CLS] when was i want to sing in opera

In [8]:
def predict(model,cands_df,inp):
    y_preds = model.predict([i for i in inp])
    y_probs = [max(y) for y in y_preds]
    y = [np.argmax(pred) for pred in y_preds]
    cands_df['predictions'] = y
    cands_df['probs'] = y_probs
    return cands_df
sa_model = mk_model(sa_weights_path)
s = predict(sa_model,sa,sa_inps)
s = s.sort_values('probs',ascending=False)
s = s.loc[s['predictions']==1]
s= s.drop_duplicates(subset='id',keep='first')
s = pd.merge(s,doc_text,left_on='id',right_on='id',how='left')
def get_span(row):
    h = row['text']
    ans = row['candidate']
    if ans.lower() == "no":
        val = "NO"
    elif ans.lower() == "yes":
        val = "YES"
    elif ans.lower() == "nan":
        val = None
    elif ans.lower() != ans.lower():
        val = None
    else:
        ans_sp = ans.split(" ")
        l = len(ans_sp)
        spl = h.split(" ")
        loop = True
        while loop:
            
            try:
                ind_1 = spl.index(ans_sp[0]) 
                for i in range(1,l):
                    if spl[ind_1+i] != ans_sp[i]:
                        #spl[ = spl[ind_1+1:]
                        for t in range(ind_1+i):
                            spl[t] = None
                        break
                    else:
                        end = ind_1 +i+1
                    if i == l-2:
                        loop = False
                        val = "{}:{}".format(ind_1,end)
                        break
                
            except:
                val =None
                loop = False
                break
                
        
        
    return val 


In [9]:
s['sa_span'] = ""
s['sa_span'] = s.apply(get_span,axis=1)

In [ ]:

p = pd.merge(p,s[['id','sa_span']],left_on='id',right_on='id',how='left')
d = pd.read_csv(sub_path)
for i in d.index:
    if 'long' in d.loc[i,'example_id']:
        h = int(d.loc[i,'example_id'].split("_")[0])
        try:
            ans = str(p.loc[p['id']==d.loc[i,'example_id'].split("_")[0]].span.values[0])
            d.loc[i,'PredictionString'] = ans
        except:
            d.loc[i,'PredictionString'] = None
    if 'short' in d.loc[i,'example_id']:
        h = int(d.loc[i,'example_id'].split("_")[0])
        try:
            ans = str(p.loc[p['id']==d.loc[i,'example_id'].split("_")[0]].sa_span.values[0])
            d.loc[i,'PredictionString'] = ans
        except:
            d.loc[i,'PredictionString'] = None
for i in d.index:
    if d.loc[i,'PredictionString'] == "nan":
        d.loc[i,'PredictionString'] = ""
    elif d.loc[i,'PredictionString'] == "None":
        d.loc[i,'PredictionString'] = ""
    
d.to_csv('submission.csv',index=False)